In [ ]:
import openai
import requests
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from amadeus import Client, ResponseError
import pandas as pd

In [19]:
flight_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'
params= {
    'originLocationCode': 'CLE',  # Cleveland
    'destinationLocationCode': 'NYC',  # New York
    'departureDate': '2025-05-01',
    'adults': 1,
    'nonStop': 'false',
    'currencyCode': 'USD',
    'max': 10  # Limit results
}

hotels_url = 'https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city'
params= {
    'cityCode': 'CLE',  # IATA code for Cleveland
    'radius': 5,
    'radiusUnit': 'KM',
    'ratings': '3,4,5',  # Optional: filter by star ratings
    'hotelSource': 'ALL'  # ALL, BEDBANK, or DIRECTCHAIN
}


# For authentication
auth_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': 'RW5x5Bn2aMZYmgyHCyomxPetDteMmM2a',
    'client_secret': 'sKvaa9jS36eO7OCL'
}

# Get access token
auth_response = requests.post(auth_url, data=auth_data)
access_token = auth_response.json()['access_token']

# Use token in API request
headers = {
    'Authorization': f'Bearer {access_token}'
}

response_hotel = requests.get(hotels_url, headers=headers, params=params)
respone_flight = requests.get(flight_url, headers=headers, params=params)
hotel_data = response_hotel.json()
flight_data = respone_flight.json()


In [21]:
flights_list = []
for offer in flight_data['flight_data']:
        # Extract the most important data from each flight offer
        flight_info = {
            'offer_id': offer.get('id', 'ID not available'),
            'price': offer.get('price', {}).get('total', 'Price not available'),
            'currency': offer.get('price', {}).get('currency', 'Currency not available'),
            # Get the first itinerary segment information (typically the outbound flight)
            'departure_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('iataCode', 'Airport not available'),
            'arrival_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[-1].get('arrival', {}).get('iataCode', 'Airport not available'),
            'departure_date': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('at', 'Date not available'),
            'airline': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('carrierCode', 'Airline not available')
        }
        flights_list.append(flight_info)
flight_df = pd.DataFrame(flights_list)
print(flights_list)

KeyError: 'flight_data'

In [ ]:
hotels_list = []
for hotel in hotel_data['data']:
    hotel_info = {
        'name': hotel.get('name', 'Name not available'),
        'hotel_id': hotel.get('hotelId', 'ID not available'),
        'rating': hotel.get('rating', 'Rating not available'),
    }
    hotels_list.append(hotel_info)
df = pd.DataFrame(hotels_list)
print(df)

                            name  hotel_id  rating
0     DBLTREE CLEVELAND DOWNTOWN  DTCLE216       3
1  WYNDHAM PLAYHOUSESQUARE HOTEL  WYCLEPHS       5
2     RADISSON CLEVELAND GATEWAY  RDCLE627       3
3     THE RITZ-CARLTON CLEVELAND  RZCLECLE       5
4           COMFORT INN DOWNTOWN  CICLE167       3
5    HILTON GI CLEVELAND GATEWAY  GICLE246       3
6            UNIVERSITY HOTEL AN  ILCLE101       3
7  INTERCONTINENTAL SUITES HOTEL  ICCLEICB       3
